In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import pyvista as pv
import os
import utils
from stpyvista import stpyvista
from datetime import datetime
import importlib

/opt/homebrew/lib/python3.11/site-packages/stpyvista/__init__.py:18: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("vtk", sizing_mode="stretch_both")


In [2]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [3]:
df_grouped = utils.get_full_training_dataset(df)

In [4]:
df_grouped

tmp     hum    CO2    VOC    vis    IR  \
device_id     date_time_rounded                                                
hka-aqm-am001 2022-09-05 13:00:00  25.050  51.890  993.0  606.5   68.0  11.5   
              2022-09-05 13:30:00  25.110  51.935  864.0  497.0  109.0  18.5   
              2022-09-05 14:00:00  25.345  51.760  658.0  491.5   95.5  14.0   
              2022-09-05 14:30:00  25.580  51.630  642.0  579.0  116.0  19.0   
              2022-09-05 16:00:00  25.710  50.270  617.0  633.0   25.0   9.0   
...                                   ...     ...    ...    ...    ...   ...   
hka-aqm-am308 2023-09-26 22:00:00  27.435  37.415  522.5  957.5    4.0   1.0   
              2023-09-26 22:30:00  27.380  37.510  527.5  958.5    5.5   2.0   
              2023-09-26 23:00:00  27.370  37.580  524.5  949.0    5.5   2.5   
              2023-09-26 23:30:00  27.345  37.630  527.0  955.0    4.0   0.0   
              2023-09-27 00:00:00  27.330  37.650  525.0  951.0    4.0   0.0   

                                   WIFI   BLE   rssi  channel_rssi  ...  \
device_id     date_time_rounded                                     ...   
hka-aqm-am001 2022-09-05 13:00:00   1.0   0.5 -127.5        -127.5  ...   
              2022-09-05 13:30:00   2.0   0.0 -128.5        -128.5  ...   
              2022-09-05 14:00:00   2.5   0.5 -134.0        -134.0  ...   
              2022-09-05 14:30:00   0.0   0.0 -134.0        -134.0  ...   
              2022-09-05 16:00:00   1.0   0.0 -138.0        -138.0  ...   
...                                 ...   ...    ...           ...  ...   
hka-aqm-am308 2023-09-26 22:00:00   3.5  13.5  -74.0         -74.0  ...   
              2023-09-26 22:30:00   1.0   0.0  -73.0         -73.0  ...   
              2023-09-26 23:00:00   1.5   0.0  -72.5         -72.5  ...   
              2023-09-26 23:30:00   3.0  13.5  -75.0         -75.0  ...   
              2023-09-27 00:00:00   0.0   0.0  -70.0         -70.0  ...   

                                   VOC_lag_3  VOC_lag_4  VOC_lag_5  VOC_next  \
device_id     date_time_rounded                                                
hka-aqm-am001 2022-09-05 13:00:00        NaN        NaN        NaN     497.0   
              2022-09-05 13:30:00        NaN        NaN        NaN     491.5   
              2022-09-05 14:00:00        NaN        NaN        NaN     579.0   
              2022-09-05 14:30:00      606.5        NaN        NaN       NaN   
              2022-09-05 16:00:00      579.0      491.5      497.0       NaN   
...                                      ...        ...        ...       ...   
hka-aqm-am308 2023-09-26 22:00:00      953.0      953.5      944.0     958.5   
              2023-09-26 22:30:00      950.5      953.0      953.5     949.0   
              2023-09-26 23:00:00      955.0      950.5      953.0     955.0   
              2023-09-26 23:30:00      957.5      955.0      950.5     951.0   
              2023-09-27 00:00:00      958.5      957.5      955.0       NaN   

                                   vis_lag_1  vis_lag_2  vis_lag_3  vis_lag_4  \
device_id     date_time_rounded                                                 
hka-aqm-am001 2022-09-05 13:00:00        NaN        NaN        NaN        NaN   
              2022-09-05 13:30:00       68.0        NaN        NaN        NaN   
              2022-09-05 14:00:00      109.0       68.0        NaN        NaN   
              2022-09-05 14:30:00       95.5      109.0       68.0        NaN   
              2022-09-05 16:00:00        NaN        NaN      116.0       95.5   
...                                      ...        ...        ...        ...   
hka-aqm-am308 2023-09-26 22:00:00        4.0        5.5        5.5        5.5   
              2023-09-26 22:30:00        4.0        4.0        5.5        5.5   
              2023-09-26 23:00:00        5.5        4.0        4.0        5.5   
              2023-09-26 23:30:00        5.5        5.5        4.0        4.0   
        

In [5]:
importlib.reload(utils)
train_dataset, test_dataset, train_loader, test_loader, scaler, y_test, full_preprocessed_df, y_feature_scaler_index = utils.get_data_for_multivarate_sequential_forecast(df)
full_preprocessed_df

training data cutoff:  2023-07-14 05:00:00


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_s

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

,date_time_rounded,tmp,hum,CO2,VOC,vis,IR,WIFI,BLE,rssi,...,device_id_hka-aqm-am201a,device_id_hka-aqm-am201b,device_id_hka-aqm-am204,device_id_hka-aqm-am205,device_id_hka-aqm-am209,device_id_hka-aqm-am210,device_id_hka-aqm-am211,device_id_hka-aqm-am301,device_id_hka-aqm-am307,device_id_hka-aqm-am308
5,2022-10-10 14:00:00,-0.679078,2.182790,1.069387,-1.150565,0.381926,0.024891,-0.573072,-0.397273,1.533102,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,-0.233183
6,2022-10-10 14:30:00,-0.178507,1.664413,3.010989,-1.118106,0.078834,-0.057464,-1.259171,-0.459544,0.924104,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,-0.233183
7,2022-10-10 15:00:00,0.187987,1.078309,1.217142,-0.998985,0.009453,-0.079465,-0.791828,-0.454355,0.312296,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,-0.233183
8,2022-10-10 15:30:00,0.300901,0.799584,0.745733,-0.956880,-0.025781,-0.091677,-1.557475,-0.397273,1.672181,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,-0.233183
9,2022-10-10 16:00:00,0.342721,0.814692,0.654266,-0.835616,-0.035219,-0.098338,-1.885609,-0.568519,1.440382,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,-0.233183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200731,2023-09-26 22:00:00,0.936563,0.196287,0.228591,0.558919,-0.229635,-0.154957,0.411330,0.587394,0.837706,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,4.288478
200732,2023-09-26 22:30:00,0.921229,0.206185,0.263771,0.562287,-0.227747,-0.151626,-1.229341,-0.568519,0.884066,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,4.288478
200733,2023-09-26 23:00:00,0.918441,0.213479,0.242663,0.530287,-0.227747,-0.149961,-0.901206,-0.568519,0.907246,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,4.288478
200734,2023-09-26 23:30:00,0.911471,0.218689,0.260253,0.550498,-0.229635,-0.158287,0.083196,0.587394,0.791346,...,-0.062721,-0.090009,-0.064986,-0.234326,-0.232945,-0.234163,-0.228308,-0.233807,-0.226548,4.288478


In [6]:
importlib.reload(utils)
model, scaler, rmse, mae, mape = utils.create_multivariate_transformer_model_for_feature(df, d_model=128, nhead=8, num_layers=4, dropout=0.1, batch_size=128, learning_rate=0.00025, epochs=2)

training data cutoff:  2023-07-15 02:45:00


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_s

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

ValueError: non-broadcastable output operand with shape (62745,1) doesn't match the broadcast shape (62745,49)

In [7]:
importlib.reload(utils)
model, scaler, rmse, mae, mape = utils.create_multivariate_transformer_model_for_feature(df, d_model=128, nhead=8, num_layers=4, dropout=0.1, batch_size=128, learning_rate=0.00025, epochs=2)

training data cutoff:  2023-07-15 02:45:00


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:868: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_count = x[0].shape[1]
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:869: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  print("feature count: ", feature_count)
/Users/timehmann/Library/Mob

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
importlib.reload(utils)
model, scaler = utils.create_multivariate_model_for_feature(df, aggregation_level='half_hour')

Training data shape: torch.Size([150050, 47]) torch.Size([150050])
Testing data shape: torch.Size([50023, 47]) torch.Size([50023])
     actual  predicted
0     408.0  477.67099
1     410.0  477.67099
2     406.5  477.67099
3     410.5  477.67099
4     413.5  477.67099
..      ...        ...
123   460.0  477.67099
124   440.0  477.67099
125   432.0  477.67099
126   432.5  477.67099
127   423.5  477.67099

[128 rows x 2 columns]
Epoch 1/1000, Validation Loss: 1.3423


KeyboardInterrupt: 

In [ ]:
train.columns

Index(['tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI', 'BLE', 'rssi',
       'channel_rssi', 'channel_index', 'spreading_factor', 'bandwidth',
       'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month', 'hour_sin',
       'semester_SS23', 'semester_WS22/23', 'semester_WS23/24', 'tmp_lag_1',
       'tmp_lag_2', 'tmp_lag_3', 'tmp_lag_4', 'tmp_lag_5', 'hum_lag_1',
       'hum_lag_2', 'hum_lag_3', 'hum_lag_4', 'hum_lag_5', 'CO2_lag_1',
       'CO2_lag_2', 'CO2_lag_3', 'CO2_lag_4', 'CO2_lag_5', 'CO2_next',
       'VOC_lag_1', 'VOC_lag_2', 'VOC_lag_3', 'VOC_lag_4', 'VOC_lag_5',
       'vis_lag_1', 'vis_lag_2', 'vis_lag_3', 'vis_lag_4', 'vis_lag_5',
       'CO2_next_scaled'],
      dtype='object')

In [50]:
from datetime import datetime

importlib.reload(utils)
start_time = datetime.strptime('2022-11-03 10:00:00', '%Y-%m-%d %H:%M:%S')
predictions = utils.predict_data_multivariate_transformer(start_time=start_time, selected_room='am001', prediction_count=5)
predictions

loading latest scaler: models/transformer_multivariate_quarter_hour_CO2_scaler_v1.pth
loading latest model: models/transformer_multivariate_quarter_hour_CO2_model_v1.pth


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid f

,date_time_rounded,CO2_pred
0,2022-11-03 00:00:00,458.0
1,2022-11-03 00:15:00,457.0
2,2022-11-03 00:30:00,463.0
3,2022-11-03 00:45:00,462.0
4,2022-11-03 01:00:00,459.0
...,...,...
91,2022-11-03 22:45:00,461.0
92,2022-11-03 23:00:00,461.0
93,2022-11-03 23:15:00,461.0
94,2022-11-03 23:30:00,461.0


In [21]:
predictions.columns

Index(['date_time_rounded', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI',
       'BLE', 'rssi', 'channel_rssi', 'channel_index', 'spreading_factor',
       'bandwidth', 'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month',
       'hour_sin', 'semester_SS23', 'semester_WS22/23', 'semester_WS23/24',
       'group', 'device_id_hka-aqm-am001', 'device_id_hka-aqm-am002',
       'device_id_hka-aqm-am003a', 'device_id_hka-aqm-am003b',
       'device_id_hka-aqm-am004', 'device_id_hka-aqm-am005',
       'device_id_hka-aqm-am107', 'device_id_hka-aqm-am109',
       'device_id_hka-aqm-am110', 'device_id_hka-aqm-am111',
       'device_id_hka-aqm-am115', 'device_id_hka-aqm-am116',
       'device_id_hka-aqm-am117', 'device_id_hka-aqm-am123',
       'device_id_hka-aqm-am124', 'device_id_hka-aqm-am126',
       'device_id_hka-aqm-am201a', 'device_id_hka-aqm-am201b',
       'device_id_hka-aqm-am204', 'device_id_hka-aqm-am205',
       'device_id_hka-aqm-am209', 'device_id_hka-aqm-am210',
       'devi